# Data Cleaning Notebook

## Objectives

* Assess and handle missing values 
* Clean data

## Inputs

* outputs/datasets/collection/HousePrices.csv

## Outputs

* Cleaned data in outputs/datasets/cleaned
* Data cleaning pipline

## Conclusions

* Drop ['EnclosedPorch', 'WoodDeckSF'] because each variable has more than 80% missing values.
* Nine out of 24 variables (75% of the columns) have missing values.


---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chdir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Load Data

In [ ]:
import pandas as pd
df = pd.read_csv("outputs/datasets/collection/HousePrices.csv")
df.head(5)

# Data Exploration

In the quick display of the first five rows of our dataframe object above, we already see four columns and several cells with missing values. Here we dig deeper and get all variables that have missing values.

 * Nine out of 24 variables (which is 75% of the columns) have missing values. 

In [ ]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

In [ ]:
len(vars_with_missing_data)

In [ ]:
df[vars_with_missing_data].info()

### Profile Report

* We generate pandas profiling report for each variable with missing values.
  * We import pandas_profiling library and generate a profile report.
  * Some variable have many zeros.

In [ ]:
# Code from walkthrough project 02, data cleaning notebook
from pandas_profiling import ProfileReport
if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

---

# Data Cleaning

## Assessing Missing Data Levels

* Custom function to display missing data levels in a dataframe, it shows the aboslute levels, relative levels and data type

In [ ]:
def EvaluateMissingData(df):
  missing_data_absolute = df.isnull().sum()
  missing_data_percentage = round(missing_data_absolute/len(df)*100 , 2)
  df_missing_data = (pd.DataFrame(
                          data= {"RowsWithMissingData": missing_data_absolute,
                                 "PercentageOfDataset": missing_data_percentage,
                                 "DataType":df.dtypes}
                                  )
                    .sort_values(by=['PercentageOfDataset'],ascending=False)
                    .query("PercentageOfDataset > 0")
                    )

  return df_missing_data

Check missing data levels for collected dataset

* Two variables, namely ['EnclosedPorch', 'WoodDeckSF'], have more than 85% missing values each.

In [ ]:
EvaluateMissingData(df)

## Handling Missing Data

* Defining DataCleaningEffect() function
  * Code copied and adapted from Unit 9 of the feature engine lesson

In [ ]:
import seaborn as sns
sns.set(style="whitegrid")
import matplotlib.pyplot as plt

def DataCleaningEffect(df_original,df_cleaned,variables_applied_with_method):

  flag_count=1 # Indicate plot number
  
  # distinguish between numerical and categorical variables
  categorical_variables = df_original.select_dtypes(exclude=['number']).columns 

  # scan over variables, 
    # first on variables that you applied the method
    # if the variable is a numerical plot, a histogram if categorical plot a barplot
  for set_of_variables in [variables_applied_with_method]:
    print("\n=====================================================================================")
    print(f"* Distribution Effect Analysis After Data Cleaning Method in the following variables:")
    print(f"{set_of_variables} \n\n")
  

    for var in set_of_variables:
      if var in categorical_variables:  # it is categorical variable: barplot
        
        df1 = pd.DataFrame({"Type":"Original","Value":df_original[var]})
        df2 = pd.DataFrame({"Type":"Cleaned","Value":df_cleaned[var]})
        dfAux = pd.concat([df1, df2], axis=0)
        fig , axes = plt.subplots(figsize=(15, 5))
        sns.countplot(hue='Type', data=dfAux, x="Value",palette=['#432371',"#FAAE7B"])
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.xticks(rotation=90)
        plt.legend() 

      else: # it is numerical variable: histogram

        fig , axes = plt.subplots(figsize=(10, 5))
        sns.histplot(data=df_original, x=var, color="#432371", label='Original', kde=True,element="step", ax=axes)
        sns.histplot(data=df_cleaned, x=var, color="#FAAE7B", label='Cleaned', kde=True,element="step", ax=axes)
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.legend() 

      plt.show()
      flag_count+= 1

### Data Cleaning Summary
* Drop ['EnclosedPorch', 'WoodDeckSF'] because each has more than 80% missing values.
* Imputation
   * Mean- ['LotFrontage', 'BedroomAbvGr' ]
   * Median- ['2ndFlrSF', 'GarageYrBlt', 'MasVnrArea' ]
   * Categorical- ['GarageFinish','BsmtFinType1' ] 

### Split Train and Test Set

In [ ]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['SalePrice'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

Check the number of variables in the train set with missing values to make sure that the split represents the whole dataset.

In [ ]:
df_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

### 1. Drop variables
* Variables to drop: ['EnclosedPorch', 'WoodDeckSF']
* We create a separate data frame applying the imputation method and assess the effect of this data cleaning step.
* List columns of the new data frame to see if the variables have beed dropped.

In [ ]:
from feature_engine.selection import DropFeatures

variables_to_drop=['EnclosedPorch' , 'WoodDeckSF']
imputer = DropFeatures(features_to_drop=variables_to_drop)
df_method = imputer.fit_transform(TrainSet)

for i in variables_to_drop:
    print(i in df_method.columns.to_list())

### 2. Mean Imputation
* Variables: ['LotFrontage' , 'BedroomAbvGr']

In [ ]:
from feature_engine.imputation import MeanMedianImputer

variables_mean = ['LotFrontage' , 'BedroomAbvGr']
imputer = MeanMedianImputer(imputation_method='mean', variables=variables_mean)
df_method = imputer.fit_transform(TrainSet)
DataCleaningEffect(df_original=TrainSet,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_mean)

### 3. Median Imputation
* Variables: ['2ndFlrSF' , 'GarageYrBlt', 'MasVnrArea']

In [ ]:
variables_median = ['2ndFlrSF' , 'GarageYrBlt', 'MasVnrArea']
imputer = MeanMedianImputer(imputation_method='median', variables=variables_median)
df_method = imputer.fit_transform(TrainSet)
DataCleaningEffect(df_original=TrainSet,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_median)

Looking at the effect of median imputation on GarageYrBlt, most of the imputed values are concentrated around 1975. 
We inspect if the missing values indicate that there is no garage at all.
* The below data shows where GarageYrBlt is NaN, GarageArea is zero.
* Intuitively, the size of the garage is likely to have more effect on sales price than its age. So we can consider dropping GarageYrBlt.

In [ ]:
TrainSet[(TrainSet['GarageArea'] ==0)][['GarageYrBlt', 'GarageArea']]

### 4. Categorical Imputation
* Variables: ['GarageFinish' , 'BsmtFinType1']

In [ ]:
from feature_engine.imputation import CategoricalImputer

variables_categorical = ['GarageFinish' , 'BsmtFinType1']
imputer = CategoricalImputer(imputation_method='missing', fill_value='None', variables=variables_categorical)
df_method = imputer.fit_transform(TrainSet)
DataCleaningEffect(df_original=TrainSet,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_categorical)

In [ ]:
TrainSet[(TrainSet['GarageArea'] ==0)][['GarageFinish', 'GarageArea']]

In [ ]:
TrainSet[(TrainSet['TotalBsmtSF'] ==0)][['BsmtFinType1', 'TotalBsmtSF']]

### Data Cleaning Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

dataCleaning_pipeline = Pipeline([
      ( 'mean',  MeanMedianImputer(imputation_method='mean',
                                     variables=['LotFrontage' , 'BedroomAbvGr']) ),
      ( 'median',  MeanMedianImputer(imputation_method='median',
                                     variables=['2ndFlrSF', 'MasVnrArea']) ),
      ( 'categorical',  CategoricalImputer(imputation_method='missing',
                                     fill_value='None',
                                     variables=['GarageFinish' , 'BsmtFinType1']) ),
      ( 'drop',  DropFeatures(features_to_drop=['EnclosedPorch', 'GarageYrBlt', 'WoodDeckSF']) )
])

Apply the pipeline to the whole dataset to get cleaned data.

In [ ]:
TrainSet, TestSet = dataCleaning_pipeline.fit_transform(TrainSet) , dataCleaning_pipeline.fit_transform(TestSet)

In [ ]:
df = dataCleaning_pipeline.fit_transform(df)

In [ ]:
EvaluateMissingData(TrainSet)

In [ ]:
EvaluateMissingData(TestSet)

In [ ]:
EvaluateMissingData(df)

---

## Push files to Repo

In [ ]:
try:
  os.makedirs(name='outputs/datasets/cleaned')
except Exception as e:
  print(e)

### Train Set

In [ ]:
TrainSet.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)

### Test Set

In [ ]:
TestSet.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv", index=False)

### Cleaned Dataset

In [ ]:
df.to_csv("outputs/datasets/cleaned/HousePricesCleaned.csv", index=False)

Save the data cleaning pipeline for later use to clean inheritted houses data

In [ ]:
import joblib

file_path = f'outputs/ml_pipeline/data_cleaning'

try:
  os.makedirs(name=file_path)
except Exception as e:
  print(e)

In [ ]:
joblib.dump(value=dataClean_pipeline, filename=f"{file_path}/dataCleaning_pipeline.pkl")